<a href="https://colab.research.google.com/github/parksungjun11/parksungjun11/blob/main/Individual_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**반드시 런타임 유형을 GPU로 변경하고 실행해주세요!**

1. 한글 폰트를 설치하고 런타임 다시 시작을 눌러주세요 이 과정이 끝나면 데이터 업로드부분으로 바로 넘어가시면 됩니다.

In [ ]:
# 한글 폰트 설치하기 (꼭! 설치가 완료되면 [런타임 다시 시작]을 누르고 다시 실행하기)
!apt install fonts-nanum -y

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트 설정하기
font_path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font1 = fm.FontProperties(fname=font_path, size=10)
plt.rc('font', family='NanumBarunGothic')
matplotlib.font_manager._rebuild()

In [ ]:
#코드를 실행하는데 필요한 라이브러리
import os
import zipfile
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib.image as mpimg
import shutil
from google.colab import files 
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
import numpy as np
import time
from PIL import Image
import io

2. 이미 한번 시행하여 사용할 데이터셋을 만들었기 때문에 데이터셋을 만들고 각자 강아지별로 이미지를 다운받아와서 또 다시 데이터 셋을 안만드셔도 됩니다. (이러한 코드를 사용하여 데이터를 만들었다를 보는것이 목적이기 때문입니다.)

In [ ]:
#bing 이미지를 다운받기 위한 코드 
!git clone https://github.com/ndb796/bing_image_downloader

In [ ]:
#수집한 bing 이미지들을 넣을 데이터셋 만들기
from bing_image_downloader.bing_image_downloader import downloader
directorylist = [
    './dog_dataset/training/',
    './dog_dataset/testing/',
]

# 초기 디렉토리 만들기
for directory in directorylist:
    if not os.path.isdir(directory):
        os.makedirs(directory)

# 수집한 이미지를 학습 데이터와 평가 데이터로 구분하는 함수
def data_split(dog_name, training_count):
    # 학습 및 평가 데이터셋 디렉토리 만들기
    for directory in directorylist:
        if not os.path.isdir(directory + '/' + dog_name):
            os.makedirs(directory + '/' + dog_name)
    # 학습 및 평가 데이터셋 준비하기
    count = 0
    for file_name in os.listdir(dog_name):
        if count < training_count:
            print(f'[Train Dataset] {file_name}')
            shutil.move(dog_name + '/' + file_name, './dog_dataset/training/' + dog_name + '/' + file_name)
        else:
            print(f'[Test Dataset] {file_name}')
            shutil.move(dog_name + '/' + file_name, './dog_dataset/testing/' + dog_name + '/' + file_name)
        count += 1
    shutil.rmtree(dog_name)

3. 2번에서 언급했던 것처럼 bing에서 비숑, 말티즈, 푸들 ,포메라니안, 시츄의 이미지를 가져옵니다.(코드를 채점하실 때는 이 부분은 참고만 해주시면 될거같습니다.)

In [ ]:
#bing에서 비숑의 이미지를 검색하여 약 300여개의 이미지를 가져오고 8:2의 비율로 random하게 train과 test에 저장
dog_name = '비숑'
downloader.download(dog_name, limit=300,  output_dir='./', adult_filter_off=True, force_replace=False, timeout=60)
data_split(dog_name, 230)

In [ ]:
#bing에서 말티즈의 이미지를 검색하여 약 300여개의 이미지를 가져오고 8:2의 비율로 random하게 train과 test에 저장
dog_name = '말티즈'
downloader.download(dog_name, limit=300,  output_dir='./', adult_filter_off=True, force_replace=False, timeout=60)
data_split(dog_name, 230)

In [ ]:
#bing에서 푸들의 이미지를 검색하여 약 300여개의 이미지를 가져오고 8:2의 비율로 random하게 train과 test에 저장
dog_name = '푸들'
downloader.download(dog_name, limit=300,  output_dir='./', adult_filter_off=True, force_replace=False, timeout=60)
data_split(dog_name, 230)

In [ ]:
#bing에서 포메라니안의 이미지를 검색하여 약 300여개의 이미지를 가져오고 8:2의 비율로 random하게 train과 test에 저장
dog_name = '포메라니안'
downloader.download(dog_name, limit=300,  output_dir='./', adult_filter_off=True, force_replace=False, timeout=60)
data_split(dog_name, 230)

In [ ]:
#bing에서 시츄의 이미지를 검색하여 약 300여개의 이미지를 가져오고 8:2의 비율로 random하게 train과 test에 저장
dog_name = '시츄'
downloader.download(dog_name, limit=300,  output_dir='./', adult_filter_off=True, force_replace=False, timeout=60)
data_split(dog_name, 230)

In [ ]:
#colab에 있는 데이터셋을 zip으로 압축 (매번 코드를 실행할때 마다 데이터셋이 바뀌고 데이터 수집에 시간이 걸리므로 좀 더 시간단축)
dog_zip = zipfile.ZipFile('/content/dog_dataset.zip', 'w') # 압축 후 저장 위치
for folder, subfolders, files in os.walk('/content/dog_dataset'): # 압축 할 폴더
    for file in files:
        if file.endswith('.jpg'): # 압축할 파일 확장자
            dog_zip.write(os.path.join(folder, file), os.path.relpath(os.path.join(folder,file), '/content/dog_dataset/'), 
                          compress_type = zipfile.ZIP_DEFLATED)
dog_zip.close()

4. 한글 폰트 코드가 완료되면 colab에 데이터셋 업로드하는 부분으로 바로 넘어오시면 됩니다.

In [ ]:
#google colab에 데이터셋 업로드 (대략 데이터 크기가 커서 30분정도 소요)
from google.colab import files 
uploaded = files.upload()
for fn in uploaded.keys():
  print('업로드 파일 "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

In [ ]:
#업로드한 데이터셋 압축 풀기
!mkdir -p ./dog_dataset
!unzip dog_dataset.zip -d dog_dataset

5. 압축을 푼 데이터셋트의 기본정보 확인 및 train에 있는 각각 클래스 (비숑, 말티즈, 푸들, 포메라니안, 시츄)의 이미지를 확인합니다.

In [ ]:
#업로드한 데이터셋의 기본정보를 얻기 위해 실시
# 기본 경로
base_dir = '/content/dog_dataset'

train_dir = os.path.join(base_dir, 'training')
test_dir = os.path.join(base_dir, 'testing')

# 훈련에 사용되는 고양이/개 이미지 경로
train_말티즈_dir = os.path.join(train_dir, '말티즈')
train_비숑_dir = os.path.join(train_dir, '비숑')
train_시츄_dir = os.path.join(train_dir, '시츄')
train_포메라니안_dir = os.path.join(train_dir, '포메라니안')
train_푸들_dir = os.path.join(train_dir, '푸들')
print(train_말티즈_dir)
print(train_비숑_dir)
print(train_시츄_dir)
print(train_포메라니안_dir)
print(train_푸들_dir)

# 테스트에 사용되는 고양이/개 이미지 경로
test_말티즈_dir = os.path.join(test_dir, '말티즈')
test_비숑_dir = os.path.join(test_dir, '비숑')
test_시츄_dir = os.path.join(test_dir, '시츄')
test_포메라니안_dir = os.path.join(test_dir, '포메라니안')
test_푸들_dir = os.path.join(test_dir, '푸들')

print(test_말티즈_dir)
print(test_비숑_dir)
print(test_시츄_dir)
print(test_포메라니안_dir)
print(test_푸들_dir)

In [ ]:
#위에 처럼 각각 클래스별로 train,test에 어느정도의 이미지가 있는지 확인
train_말티즈_fname = os.listdir( train_말티즈_dir )
train_비숑_fname = os.listdir( train_비숑_dir )
train_시츄_fname = os.listdir( train_시츄_dir )
train_포메라니안_fname = os.listdir( train_포메라니안_dir )
train_푸들_fname = os.listdir( train_푸들_dir )

print(train_말티즈_fname[:7])
print(train_비숑_fname[:7])
print(train_시츄_fname[:7])
print(train_포메라니안_fname[:7])
print(train_푸들_fname[:7])


print('train dataset에 속한 말티즈 이미지 개수 :', len(os.listdir(train_말티즈_dir)), '개')
print('train dataset에 속한 비숑 이미지 개수 :', len(os.listdir(train_비숑_dir)), '개')
print('train dataset에 속한 시츄 이미지 개수 :', len(os.listdir(train_시츄_dir)), '개')
print('train dataset에 속한 포메라니안 이미지 개수 :', len(os.listdir(train_포메라니안_dir)), '개')
print('train dataset에 속한 푸들 이미지 개수 :', len(os.listdir(train_푸들_dir)), '개')
print('test dataset에 속한 말티즈 이미지 개수 :', len(os.listdir(test_말티즈_dir)), '개')
print('test dataset에 속한 비숑 이미지 개수 :', len(os.listdir(test_비숑_dir)), '개')
print('test dataset에 속한 시츄 이미지 개수 :', len(os.listdir(test_시츄_dir)), '개')
print('test dataset에 속한 포메라니안 이미지 개수 :', len(os.listdir(test_포메라니안_dir)), '개')
print('test dataset에 속한 푸들 이미지 개수 :', len(os.listdir(test_푸들_dir)), '개')

In [ ]:
#말티즈 train dataset에 있는 클래스들의 이미지를 불러옴(5개)
%matplotlib inline
nrows, ncols = 100,100
pic_index = 0

fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

pic_index+=100

next_말티즈_pix = [os.path.join(train_말티즈_dir, fname)
                for fname in train_말티즈_fname[ pic_index-5:pic_index]]
                
for i, img_path in enumerate(next_말티즈_pix ):
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off')

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

In [ ]:
#비숑 train dataset에 있는 클래스들의 이미지를 불러옴(5개)
%matplotlib inline
nrows, ncols = 100,100
pic_index = 0

fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

pic_index+=100

next_비숑_pix = [os.path.join(train_비숑_dir, fname)
                for fname in train_비숑_fname[ pic_index-5:pic_index]]
                
for i, img_path in enumerate(next_비숑_pix ):
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off')

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

In [ ]:
#시츄 train dataset에 있는 클래스들의 이미지를 불러옴(5개)
%matplotlib inline
nrows, ncols = 100,100
pic_index = 0

fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

pic_index+=65

next_시츄_pix = [os.path.join(train_시츄_dir, fname)
                for fname in train_시츄_fname[ pic_index-5:pic_index]]
                
for i, img_path in enumerate(next_시츄_pix ):
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off')

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

In [ ]:
#포메라니안 train dataset에 있는 클래스들의 이미지를 불러옴(5개)
%matplotlib inline
nrows, ncols = 100,100
pic_index = 0

fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

pic_index+=60

next_포메라니안_pix = [os.path.join(train_포메라니안_dir, fname)
                for fname in train_포메라니안_fname[ pic_index-5:pic_index]]
                
for i, img_path in enumerate(next_포메라니안_pix ):
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off')

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

In [ ]:
#푸들 train dataset에 있는 클래스들의 이미지를 불러옴(5개)
%matplotlib inline
nrows, ncols = 100,100
pic_index = 0

fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

pic_index+=12

next_푸들_pix = [os.path.join(train_푸들_dir, fname)
                for fname in train_푸들_fname[ pic_index-5:pic_index]]
                
for i, img_path in enumerate(next_푸들_pix ):
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off')

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

6. 이미지 정규화를 통해 격자 이미지로 시각화 및 후에 모델의 학습을 위한 준비를 합니다.

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # device 객체

# 데이터셋을 불러올 때 사용할 변형(transformation) 객체 정의
transforms_train = transforms.Compose([
    transforms.Resize((225, 225)),
    transforms.RandomHorizontalFlip(), # 데이터 증진(augmentation)
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # 정규화(normalization)
])

transforms_test = transforms.Compose([
    transforms.Resize((225, 225)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_datasets = datasets.ImageFolder(os.path.join(base_dir, 'training'), transforms_train)
test_datasets = datasets.ImageFolder(os.path.join(base_dir, 'testing'), transforms_test)

train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=4, shuffle=True, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(test_datasets, batch_size=4, shuffle=True, num_workers=2)

print('학습 데이터셋 크기:', len(train_datasets))
print('테스트 데이터셋 크기:', len(test_datasets))

class_names = train_datasets.classes
print('클래스:', class_names)

In [ ]:
def image_show(input, title):
    # torch.Tensor를 numpy 객체로 변환
    input = input.numpy().transpose((1, 2, 0))
    # 이미지 정규화 해제하기
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    input = std * input + mean
    input = np.clip(input, 0, 1)
    # 이미지 출력
    plt.imshow(input)
    plt.title(title)
    plt.show()


# 학습 데이터를 배치 단위로 불러오기
iterator = iter(train_dataloader)

# 현재 배치를 이용해 격자 형태의 이미지를 만들어 시각화
inputs, classes = next(iterator)
outputs = torchvision.utils.make_grid(inputs)
image_show(outputs, title=[class_names[i] for i in classes])

7. resnet34를 이용하여 학습모델을 만들고 학습모델을 바탕으로 train에 있는 이미지를 학습시킵니다.

In [ ]:
#resnet34을 이용하여 학습할 데이터 모델을 만듬
data_model = models.resnet34(pretrained=True)
num_features = data_model.fc.in_features
# 전이 학습(transfer learning): 모델의 출력 뉴런 수를 3개로 교체하여 마지막 레이어 다시 학습
data_model.fc = nn.Linear(num_features, 5)
data_model = data_model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(data_model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
#데이터 모델의 층의 구성
data_model

In [ ]:
#만든 모델을 학습하기 전 테스트 데이터에 대해 예측실시
data_model.eval()
start_time = time.time()

with torch.no_grad():
    running2_loss = 0.
    running2_corrects = 0

    for input, label in test_dataloader:
        input = input.to(device)
        label = label.to(device)

        output = data_model(input)
        _, preds = torch.max(output, 1)
        loss = criterion(output, label)

        running2_loss += loss.item() * input.size(0)
        running2_corrects += torch.sum(preds == label.data)

        # 한 배치의 첫 번째 이미지에 대하여 결과 시각화
        print(f'[예측 결과: {class_names[preds[0]]}] (실제 정답: {class_names[label.data[0]]})')
        image_show(input.cpu().data[0], title='예측 결과: ' + class_names[preds[0]])

    epoch2_loss = running2_loss / len(test_datasets)
    epoch2_acc = running2_corrects / len(test_datasets) * 100.
    print('[Test Phase] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch2_loss, epoch2_acc, time.time() - start_time))

In [ ]:
#모델을 학습시키기 -> 총 10번 실시 
num_epoch = 15
data_model.train()
start_time = time.time()
a,b= [],[]
# 전체 반복(epoch) 수 만큼 반복하며
for epoch in range(num_epoch):
    train_running_loss = 0.
    train_running_corrects = 0
    # 배치 단위로 학습 데이터 불러오기
    for input, label in train_dataloader:
        input = input.to(device)
        label = label.to(device)
        # 모델에 입력(forward)하고 결과 계산
        optimizer.zero_grad()
        output = data_model(input)
        _, preds = torch.max(output, 1)
        loss = criterion(output, label)
        # 역전파를 통해 기울기(gradient) 계산 및 학습 진행
        loss.backward()
        optimizer.step()
        train_running_loss += loss.item() * input.size(0)
        train_running_corrects += torch.sum(preds == label.data)
    train_epoch_loss = train_running_loss / len(train_datasets)
    train_epoch_acc = train_running_corrects / len(train_datasets) * 100.
    a.append(train_epoch_loss*100)
    b.append(train_epoch_acc)
    # 학습 과정 중에 결과 출력
    print('#{} train_Loss(손실): {:.4f} train_Acc(정확도): {:.4f}% Time: {:.4f}초'.format(epoch, train_epoch_loss, train_epoch_acc,time.time() - start_time))

In [ ]:
# 그래프 시각화를 위한 %값을 float으로 실수형으로 변경함
for i in range(15):
  b[i] = format(b[i])
  b[i] = float(b[i])
print(b)  
print(a)

In [ ]:
#train과 test의 정확도와 손실을 한눈에 볼 수 있게 그래프로 시각화
epoch_num = []
for i in range(len(a)):
  epoch_num.append(i)
plt.plot(epoch_num, b, 'b', label='Training accuracy')
plt.plot(epoch_num, a, 'b',color= 'red', label='Training loss')
plt.title('Training')
plt.legend()
plt.figure()
plt.show()

8. 학습한 모델을 바탕으로 test dataset에 있는 이미지를 정확히 분류하는지 예측합니다.

In [ ]:
#학습한 모델을 바탕으로 test 데이터 셋을 다시 예측함
data_model.eval()
start_time = time.time()
with torch.no_grad():
    running2_loss = 0.
    running2_corrects = 0
    for input, label in test_dataloader:
        input = input.to(device)
        label = label.to(device)
        output = data_model(input)
        _, preds = torch.max(output, 1)
        loss = criterion(output, label)
        running2_loss += loss.item() * input.size(0)
        running2_corrects += torch.sum(preds == label.data)
        # 한 배치의 첫 번째 이미지에 대하여 결과 시각화
        print(f'[예측 결과: {class_names[preds[0]]}] (실제 정답: {class_names[label.data[0]]})')
        image_show(input.cpu().data[0], title='예측 결과: ' + class_names[preds[0]])
    epoch2_loss = running2_loss / len(test_datasets)
    epoch2_acc = running2_corrects / len(test_datasets) * 100.
    print('[Test 단계] test_loss(손실): {:.4f} test_acc(정확도): {:.4f}% Time: {:.4f}s'.format(epoch2_loss, epoch2_acc, time.time() - start_time))

9. dataset에 없는 이미지를 바탕으로 이미지를 정확히 분류하는지 확인해봅니다.    (웹페이지에서 각각 클래스 이름을 입력했을 때 나오는 이미지를 사용)

In [ ]:
#말티즈의 이미지를 가져옴
!wget https://images.mypetlife.co.kr/content/uploads/2019/12/09152009/dog-1123026_1920.jpg -O test_image.jpg

In [ ]:
#말티즈 이미지 예측
image = Image.open('test_image.jpg')
image = transforms_test(image).unsqueeze(0).to(device)

with torch.no_grad():
    output = data_model(image)
    _, preds = torch.max(output, 1)
    image_show(image.cpu().data[0], title='예측 결과: ' + class_names[preds[0]])

In [ ]:
#비숑 이미지
!wget https://images.mypetlife.co.kr/content/uploads/2017/12/09160633/Dognews-1.jpg -O test_image1.jpg

In [ ]:
#비숑 이미지 예측
image = Image.open('test_image1.jpg')
image = transforms_test(image).unsqueeze(0).to(device)

with torch.no_grad():
    output = data_model(image)
    _, preds = torch.max(output, 1)
    image_show(image.cpu().data[0], title='예측 결과: ' + class_names[preds[0]])

In [ ]:
#시츄 이미지
!wget https://images.mypetlife.co.kr/content/uploads/2020/06/09150430/samuel-girven-0AG5oa9oGBQ-unsplash-scaled-e1593587400633.jpg -O test_image2.jpg

In [ ]:
#시츄 이미지 예측
image = Image.open('test_image2.jpg')
image = transforms_test(image).unsqueeze(0).to(device)

with torch.no_grad():
    output = data_model(image)
    _, preds = torch.max(output, 1)
    image_show(image.cpu().data[0], title='예측 결과: ' + class_names[preds[0]])

In [ ]:
#포메라니안 이미지
!wget https://images.mypetlife.co.kr/content/uploads/2018/05/09160725/pexels-photo-732456.jpeg -O test_image3.jpg

In [ ]:
#포메라니안 이미지 예측
image = Image.open('test_image3.jpg')
image = transforms_test(image).unsqueeze(0).to(device)

with torch.no_grad():
    output = data_model(image)
    _, preds = torch.max(output, 1)
    image_show(image.cpu().data[0], title='예측 결과: ' + class_names[preds[0]])

In [ ]:
#푸들 이미지
!wget https://images.mypetlife.co.kr/content/uploads/2020/05/09150723/standard-poodle-4311162_1280-1024x682.jpg -O test_image4.jpg

In [ ]:
#푸들 이미지 예측
image = Image.open('test_image4.jpg')
image = transforms_test(image).unsqueeze(0).to(device)

with torch.no_grad():
    output = data_model(image)
    _, preds = torch.max(output, 1)
    image_show(image.cpu().data[0], title='예측 결과: ' + class_names[preds[0]])

10. 마지막으로 키우는 강아지의 이미지 분류를 예측합니다. (이 이미지는 압축파일의 bichon_image폴더를 다운받으셔서 하시면 됩니다.)

In [ ]:
!pip install flask-ngrok

In [ ]:
#실제 키우는 강아지 이미지를 가져오기
from flask_ngrok import run_with_ngrok
from flask import Flask, jsonify, request
# 이미지를 읽어 결과를 반환하는 함수
def get_pred(image_bytes):
    image = Image.open(io.BytesIO(image_bytes))
    image = transforms_test(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = data_model(image)
        _, preds = torch.max(output, 1)
        image_show(image.cpu().data[0], title='예측 결과: ' + class_names[preds[0]])
    return class_names[preds[0]]
application = Flask(__name__)
@application.route('/', methods=['POST'])
def image_predict():
    if request.method == 'POST':
        # 이미지 바이트 데이터 받아오기
        file = request.files['file']
        image_bytes = file.read()

        # 분류 결과 확인 및 클라이언트에게 결과 반환
        class_name = get_pred(image_bytes=image_bytes)
        print("결과:", {'이름': class_name})
        return jsonify({'이름': class_name})

In [ ]:
run_with_ngrok(application)
application.run()

아래의 주석은 10번 부분을 수행하실때 참고하서셔 하시된 될거같습니다.

In [ ]:
'''
1. 명령 프롬프트 열기 -> windows키 + r 키를 동시에 누르면 열림 -> C:\Users\user > 라고 되어있음
2. 현재 사진이 있는 경로를 C:\Users\user> 옆에 복사하여 적어줌
   ex)현재 사진이 있는 경로가 "C:\Users\user\Pictures\test2"라면, 
   C:\Users\user> cd "C:\Users\user\Pictures\test2"라고 입력을 해줌(반드시 경로 앞에 cd도 같이 입력!!!)
   성공적으로 입력이 완료되면 "C:\Users\user\Pictures\test2" > 라는 표시로 바뀌게 됨
4."C:\Users\user\Pictures\test2"> 옆에 curl -X POST -F file=@bis1.jpg http://63dd-34-71-213-192.ngrok.io 라고 입력을 함 (여기서 bis1.jpg는 사진의 이름에다가 jpg를 붙여준 것)
  http://63dd-34-71-213-192.ngrok.io는 run_with_ngrok(application) application.run()코드를 실행시켜줬을 때 두번째 running on의 주소를 복사해 주면 됨 (반드시 두번째 주소!!)
5. C:\Users\user\Pictures\test2 > curl -X POST -F file=@bis3.jpg http://63dd-34-71-213-192.ngrok.io 이런식으로 입력이 완료되면 colab에 사진이 업로드 되고 사진의 예측결과를 확인하면 끝!!
'''

In [ ]:
"""
아래는 명령 프롬프트를 열었을 때 제가 작성해본것 입니다.
1. C:\Users\user> (이는 명령 프롬프트를 처음 열었을 때 나오는 코드로 이 코드 옆에 현재 컴퓨터에 이미지가 저장된 경로를 복사해서 붙여넣어주면 됩니다.  이를 올바르게 수행하면 2번처럼 될 것입니다.)
2. C:\Users\user> cd "C:\Users\user\Pictures\test2" (저는 user의 pictures의 test2 폴더에 사진이 있어 이렇게 경로를 복사해주었고 교수님께서는 이 폴더가 저장된 곳의 경로를 불러오시면 됩니다.)

3. "C:\Users\user\Pictures\test2" > curl -X POST -F file=@bichon1.jpg http://63dd-34-71-213-192.ngrok.io (이미지가 있는 경로에서 이제 curl -X POST -F file=@bichon1.jpg http://63dd-34-71-213-192.ngrok.io를 입력해주시면 되는데 
                                                                                                        여기서는 실행할 때 마다 http://63dd-34-71-213-192.ngrok.io의 경로가 바뀌기에 이것만 running on의 두번째 주소를 복사해주시고  
                                                                                                        curl -X POST -F file=@bichon1.jpg 이부분은 그대로 복사해주시면 됩니다. )
4. "C:\Users\user\Pictures\test2" > curl -X POST -F file=@bichon2.jpg http://63dd-34-71-213-192.ngrok.io  (3번과 마찬기지로 runnin on의 두번째 주소를 복사해주시고 curl -X POST -F file=@bichon1.jpg 옆에 빈칸 한번 띄어서 붙여주시면 되시는데
                                                                                                          여기서 주의 하실 점은 bichon2.jpg입니다. 이 부분도 사진의 이름에 맞게 바꿔주시면 됩니다. )
5. "C:\Users\user\Pictures\test2" > curl -X POST -F file=@bichon3.jpg http://63dd-34-71-213-192.ngrok.io 
6. "C:\Users\user\Pictures\test2" > curl -X POST -F file=@bichon4.jpg http://63dd-34-71-213-192.ngrok.io 
7. "C:\Users\user\Pictures\test2" > curl -X POST -F file=@bichon5.jpg http://63dd-34-71-213-192.ngrok.io 
8. "C:\Users\user\Pictures\test2" > curl -X POST -F file=@bichon6.jpg http://63dd-34-71-213-192.ngrok.io 
9. "C:\Users\user\Pictures\test2" > curl -X POST -F file=@bichon7.jpg http://63dd-34-71-213-192.ngrok.io 
10. "C:\Users\user\Pictures\test2" > curl -X POST -F file=@bichon8.jpg http://63dd-34-71-213-192.ngrok.io
11. "C:\Users\user\Pictures\test2" > curl -X POST -F file=@bichon9.jpg http://63dd-34-71-213-192.ngrok.io 

-> "C:\Users\user\Pictures\test2" 대신에 폴더가 있는 경로
   http://63dd-34-71-213-192.ngrok.io 대신에 위에 run_with_ngrok(application) application.run()코드를 실행시켜줬을 때 두번째 running on의 주소를 복사
   bichon2.jpg 이부분도 사진의 이름에 맞게 바꿔주시면 될 거 같습니다.
"""   